#Open Library Analysis - Big Data Computing Project
####Graph's Connected Components vs. k-means Clusters

##Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

##Data Aquisition

short dataset: 50000 entries

In [0]:
%sh wget -P /tmp https://raw.githubusercontent.com/attennig/BDC_datasets/main/books_short.csv

In [0]:
dbutils.fs.mv("file:/tmp/books_short.csv", "dbfs:/bdc-2020-21/datasets/books_short.csv")

long dataset: 22.589.356 entries 

remember to download it only if it is necessary

In [0]:
%sh wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G" -O /tmp/books_long.csv.bz2 && rm -rf /tmp/cookies.txt

In [0]:
dbutils.fs.ls("file:/tmp")

In [0]:
dbutils.fs.mv("file:/tmp/books_long.csv.bz2", "dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

In [0]:
dbutils.fs.ls("dbfs:/bdc-2020-21/datasets")

In [0]:
# Read dataset file into a Spark Dataframe
books_df = spark.read.load("dbfs:/bdc-2020-21/datasets/books_short.csv", 
                         format="csv", 
                         sep=";", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(books_df.count(), len(books_df.columns)))

##Data Cleaning

In [0]:
#eliminating entries with missing values or columns
columns_to_drop = []
for c in books_df.columns:
  if books_df.where(col(c).isNull()).count()/books_df.count() > 0.7:
    #remove col
    columns_to_drop += [c]

In [0]:
books_df = books_df.drop(*columns_to_drop)

In [0]:
assert "title" in books_df.columns and "key" in books_df.columns
books_df = books_df.dropna(how="any", subset=["key", "title"])
books_df = books_df.dropDuplicates(['key'])
books_df = books_df.dropDuplicates(['title'])

In [0]:
assert "subjects" in books_df.columns and "authors" in books_df.columns
books_df = books_df.na.fill({'subjects': 'unknown', 'authors': 'unknown'})

In [0]:
books_df = books_df.orderBy(["key"], ascending=[False, False])

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
# This will return a new DF with all the columns + id
books_df = books_df.withColumn("id", monotonically_increasing_id())

In [0]:
# if nltk not found
!python -m pip install nltk

!python -m pip install --upgrade pip

!python -m nltk.downloader all

In [0]:
# From Document_Clustering.ipynb
def clean_text(df, column_name, perform_stemming=True):
    """ 
    This function takes the raw text data and applies a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline
    """
    from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
    from pyspark.ml.feature import Tokenizer, StopWordsRemover
    from nltk.stem.snowball import SnowballStemmer # BE SURE NLTK IS INSTALLED ON THE CLUSTER USING THE "LIBRARIES" TAB IN THE MENU

    
    # Text preprocessing pipeline
    # 1. Text cleaning
    # 1.a Case normalization
    lower_case_df = df.select(["id",lower(col(column_name)).alias(column_name)])
    # 1.b Trimming
    trimmed_df = lower_case_df.select(["id",trim(col(column_name)).alias(column_name)])
    # 1.c Filter out punctuation symbols
    no_punct_df = trimmed_df.select(["id",(regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name)])
    # 1.d Filter out any internal extra whitespace
    cleaned_df = no_punct_df.select(["id",trim(regexp_replace(col(column_name), " +", " ")).alias(column_name)])
    # 2. Tokenization (split text into tokens)
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_df).cache()
    # 3. Stopwords removal
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    ret_df = stopwords_remover.transform(tokens_df).cache()
    # 4. Stemming (Snowball stemmer)
    if perform_stemming:
      stemmer = SnowballStemmer(language="english")
      stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
      ret_df = ret_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()
      
    return ret_df

In [0]:
clean_title_df = clean_text(books_df, "title")

In [0]:
clean_subjects_df = clean_text(books_df, "subjects")

In [0]:
clean_authors_df = clean_text(books_df, "authors", False)

##Feature Engineering

In this section we will use nlp techniques to get numerical vectors that represent text-based features

In [0]:
# final df
# |id|w2vec(clean_title)|w2vec(clean_subject)|authors|

In [0]:
RANDOM_SEED = 42 # used below to run the actual K-means clustering
EMBEDDING_SIZE = 150 # size of embedding Word2Vec vectors

In [0]:
#Word2Vec from Document_Clustering.ipynb 
def extract_w2v_features(df, column_name, out_col_name):
  from pyspark.ml.feature import Word2Vec
  
  word2vec = Word2Vec(vectorSize=EMBEDDING_SIZE, minCount=5, inputCol=column_name, outputCol=out_col_name, seed=RANDOM_SEED)
  model = word2vec.fit(df)
  features = model.transform(df).cache()
  
  return model, features

In [0]:
model, w2v_title_features = extract_w2v_features(clean_title_df, "terms_stemmed", "title_vec")

In [0]:
model, w2v_subjects_features = extract_w2v_features(clean_subjects_df, "terms_stemmed", "subjects_vec")

In [0]:
model, w2v_authors_features = extract_w2v_features(clean_authors_df, "terms", "authors_vec")

In [0]:
# Final dataframe
w2v_title_features = w2v_title_features.select(["id", "title_vec"])
w2v_subjects_features = w2v_subjects_features.select(["id", "subjects_vec"])

In [0]:
w2v_authors_features = w2v_authors_features.select(["id", "authors_vec"])

In [0]:
engineered_books_df = w2v_title_features

In [0]:
engineered_books_df = engineered_books_df.join(w2v_subjects_features, ["id"])

In [0]:
engineered_books_df = engineered_books_df.join(w2v_authors_features, ["id"])

In [0]:
engineered_books_df.show() 

+---+--------------------+--------------------+--------------------+
 id| title_vec| subjects_vec| authors_vec|
+---+--------------------+--------------------+--------------------+
 0|[-0.0233850330114...|[-1.5495021943934...|[-0.0159084685146...|
 1|[-0.0062155938940...|[0.03915416855703...|[-0.0159084685146...|
 2|[-5.9996234873930...|[0.10006115632131...|[-0.0159084685146...|
 3|[-2.4760121596045...|[0.08503663586452...|[-0.0159084685146...|
 4|[-0.0072768814861...|[0.05649676679500...|[-0.0159084685146...|
 5|[-0.0254338947124...|[-0.0152078140934...|[-0.0159084685146...|
 6|[0.03920520845401...|[0.07760113167266...|[0.00200751703232...|
 7|[-9.2583333607763...|[0.03338122158311...|[0.00200751703232...|
 8|[0.03417017988653...|[-1.5495021943934...|[0.00200751703232...|
 9|[0.11026902310550...|[-1.5495021943934...|[-0.0159084685146...|
 10|[0.00400233222171...|[-1.5495021943934...|[-0.0159084685146...|
 11|[0.01653723930940...|[-1.5495021943934...|[0.00200751703232...|
 12|[-0.0275387279689...|[-0.0139292316097...|[0.00200751703232...|
 13|[0.00126329693011...|[-1.5495021943934...|[0.00200751703232...|
 14|[0.01225566714128...|[-1.5495021943934...|[-0.0159084685146...|
 15|[0.0,0.0,0.0,0.0,...|[-1.5495021943934...|[-0.0159084685146...|
 16|[0.02947618303975...|[0.11119896988384...|[0.00200751703232...|
 17|[0.0,0.0,0.0,0.0,...|[-1.5495021943934...|[0.00200751703232...|
 18|[0.0,0.0,0.0,0.0,...|[0.10393609423190...|[0.00200751703232...|
 19|[0.03521083062514...|[0.04841202888637...|[-0.0159084685146...|
+---+--------------------+--------------------+--------------------+
only showing top 20 rows

##Graph

In [0]:
nodes_df = engineered_books_df.select(["id"])

In [0]:
# TODO
def cosine_similarity(b1, b2):
  # return float(np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y)))
  pass

In [0]:
thr = 0.5
Link = Row("src", "dst")
links = []
for row1 in df:
  for row2 in df:
    if cosine_similarity(row1, row2) > thr:
      links.append(Link(row1["id"], row2["id"]))
      links.append(Link(row2["id"], row1["id"]))
      
links_df = spark.createDataFrame(links)

In [0]:
books_graph = GraphFrame(nodes_df, links_df) 

In [0]:
books_CC = books_graph.connectedComponents()

##Clustering

In [0]:
# from Document_Clustering.ipynb  e l'altro del clustering
# k = nCC , kmeans

##Evaluation